In [18]:
#install boto3, awcli on EC2

import boto3
import os
import os.path
import shutil


# HELPER FUNCTION
def upload_dir(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            key = path + file
            s3 = boto3.resource('s3')
            s3.Bucket(bucket_name).upload_file(os.path.join(root, file), key)


bucket_name = 'alignedstorage'
prefix = "training_input"
bucket = boto3.resource('s3').Bucket(bucket_name)
ct=0
for obj in bucket.objects.filter(Prefix=prefix):
    path, file = os.path.split(obj.key)
    if path != "":
      
        # 1. grab name of the file
        file_name = str(file)
        name, _ = os.path.splitext(file_name)
  
        # 2. save file in temp
        bucket.download_file(prefix+"/"+file_name, '/tmp/'+file_name)
        
        path = '~/data/output/training_data/' + name #without extension
        os.mkdir(path)
        
        # 4. run binary, how to execute this?
        
#     #https://stackoverflow.com/questions/89228/calling-an-external-command-in-python?fbclid=IwAR0A549FtodaXDu3OnZH843qLk98Ha2mQiCeajFDb_xM88jxLNdfaxC5RdQ
    
#     ~/openpose/examples/openpose/openpose.bin
#      --video ~/ENTER PATH
#      --write_video ~/ENTER PATH FILE
#       --write_json ~/ENTER PATH --display 0

        # 5. save output video on s3
        output_prefix = 'output'
        data = 'warrior.avi' # CHANGE HERE - this should be the file data
        key = output_prefix+'/'+data
        boto3.resource('s3').Bucket(bucket_name).upload_file(data, key)
        
        # 6. save json files on s3                
        json_folder = 'json_test/' # CHANGE HERE - this should be folder from EC2
        upload_dir(json_folder, bucket_name)
        
        # 7. delete output video and json from EC2
#         os.remove() #removes a file.
#         shutil.rmtree() #deletes a directory and all its contents.
